In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import collections

In [20]:
preverbsDF = pd.read_csv('../data/WiyotPreversText.tsv',sep='\t')
preverbsDF.head()

,Wiyot,English
0,bał/wał/wáł,relating.to
1,be/bé/béh,REP
2,bi/mi/mí,soon
3,bime/mime/bige/bihge/mige,some
4,bo/bó/wo/wó/ba/wa/boh,go.to


In [35]:
preverbs = list(set(preverbsDF['English']))
len(preverbs)

77

In [3]:
kroeber = pd.read_excel(f'../data/Kroeber texts.xlsx',None)
reichard = pd.read_excel(f'../data/Reichard texts.xlsx',None)
teeter = pd.read_excel(f'../data/Teeter texts.xlsx',None)

/Users/leonjdaeim/opt/miniconda3/envs/EDA/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [11]:
def processLine(x):
    try:
        x = '\t'.join(x.dropna()).split('\t')
        return x
    except:
        return None
    

## Remove empty lines
import re
matchEndPeriod = re.compile(r'\.$')
getlowercase = lambda x: x if x.isupper() else x.lower()    
_isna = lambda x: x.isna()

def getWordPairs(phrase):
    wordpairlist = []
    for i in range(0,len(phrase)-1):
        wordpairlist.append('|'.join([phrase[i],phrase[i+1]]))
    return wordpairlist

def countFrequencyWordPairs(listofwordpairs):
    try:
        return collections.Counter(listofwordpairs[:])
    except:
        return None

In [16]:
data = crow
data = data[~data.apply(_isna,0).all(1)]
if len(data)%2 != 0:
    print('Data is not even')
else:
    print('ok to proceed')

ok to proceed


In [ ]:
crow = kroeber['Crow']
# skunk = wiyotData['Crow']
# pelican = wiyotData['Pelican']

In [116]:
processStory(kroeber,'Crow')

{'Crow': [{'wiyot': ['Kacid', 'wule', 'póyakhd', 'ił', 'šudi', 'lákhow'],
   'english_original': ['Crow',
    'please',
    'call.kin.in.law',
    'NMLZ.A',
    'ocean',
    'other.side'],
   'english_clean': ['crow',
    'please',
    'call.kin.in.law',
    'NMLZ.A',
    'ocean',
    'other.side'],
   'english_preverbs': ['please'],
   'wordPairs': ['crow|please',
    'please|call.kin.in.law',
    'call.kin.in.law|NMLZ.A',
    'NMLZ.A|ocean',
    'ocean|other.side']}]}

In [124]:
processStory(kroeber,'Crow')

Crow


dict_keys(['wiyot', 'english_original', 'english_clean', 'english_preverbs', 'wordPairs'])

In [186]:
def processStory(author,story):
    data = author[story]
    data = data[~data.apply(_isna,0).all(1)]
    #####
    wiyotList = []
    originalEnglishList = []
    processedEnglishList = []
    preverbsEnglishList = []
    pairsEnglishList = []
    ######
    for i in range(0,len(data),2):   
        ##########################################
        wiyotPhrase = data.iloc[i]
        englishPhrase = data.iloc[i+1]
        wiyotPhrase = processLine(wiyotPhrase)    
        ### Do some extra processing to the english phrase
        englishPhrase = processLine(englishPhrase)
        #englishPhrase = [getlowercase(x) for x in englishPhrahaha se]
        ### Remove EMP = empty, remove periods at the end of gloss
        englishPhraseRedux = [getlowercase(matchEndPeriod.sub('',x)) for x in englishPhrase if x !='EMP'] #Remove EMP for empty 
        englishPreverbs = [_ for _ in englishPhraseRedux if _ in preverbs] 

        ### Get word pairs
        wps = getWordPairs(englishPhraseRedux)

        wiyotList.append(wiyotPhrase)
        originalEnglishList.append(englishPhrase)
        processedEnglishList.append(englishPhraseRedux)
        preverbsEnglishList.append(englishPreverbs)
        pairsEnglishList.append(wps)
        
    outDict = {"wiyot":wiyotList,
               "english_original":originalEnglishList,
              "english_clean":processedEnglishList,
              "english_preverbs":preverbsEnglishList,
              'wordPairs':pairsEnglishList}
    return outDict

In [190]:
kroeber_processed = {story:processStory(kroeber,story) for story in kroeber.keys()}

In [191]:
reichard_processed = {story:processStory(reichard,story) for story in reichard.keys()}

In [192]:
teeter_processed = {story:processStory(teeter,story) for story in teeter.keys()}

In [212]:
def getPreverbs(dataset):
    try:
        dPrev = {k:v['english_preverbs'] for k,v in dataset.items()}
        return pd.DataFrame.from_dict(dPrev,orient='index')
    
    except:
        return []

In [213]:
teeter_preverbs =  getPreverbs(teeter_processed)
reichard_preverbs =  getPreverbs(reichard_processed)
kroeber_preverbs =  getPreverbs(kroeber_processed)

In [220]:
pd.concat([teeter_preverbs,kroeber_preverbs])

,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,200
1. Abalone,"[INCH, INCH]",[],[],[],[PROG],[],[EMPH],[3NEG],"[PROG, EMPH]",[PST],...,None,None,None,None,None,None,None,None,None,None
2. Snake,"[INCH, 1NEG]",[EMPH],"[very, EMPH]",[around],[],[PROB],"[INCH, go.to]","[EMPH, ITER]",[INCH],[PST],...,None,None,None,None,None,None,None,None,None,None
3. Owl,[],[PST],"[around, in.vain]",[],"[EMPH, 3NEG]",[ITER],"[EMPH, PROG]",[INCH],[INCH],[very],...,None,None,None,None,None,None,None,None,None,None
4. Mole,[],[PROG],"[on.arrival, PROG]",[],[PST],[PST],[PST],"[PST, PST]",[PST],[PST],...,None,None,None,None,None,None,None,None,None,None
5. Raccoon,"[ITER, INCH]","[in.vain, ITER, INCH, PROG, INCH]","[PST, ITER]","[suddenly, INCH]",[],"[PROG, suddenly, INCH]",[go.to],None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82. Stick Pictures,[],[],[around],[],"[INCH, already, enough, around]",[EMPH],[INCH],[around],[INCH],[EMPH],...,[PROG],[CESS],[go.to],[PROG],[],"[PROG, PFV]",[EMPH],[EMPH],[INCH],[]
83. Meeting,[PROG],[PROG],[PL],"[PROG, PL]","[then, ITER]","[EMPH, PROG]",[],"[EMPH, PROG, for.nothing]",[PROG],[],...,None,None,None,None,None,None,None,None,None,None
Crow,[please],"[please, PST, PFV]","[PST, PL]",[],"[PST, ITER, all]","[PROG, all]",[],[ITER],[],[if],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pelican,[PROG],"[PL, INCH]",[PROG],"[PST, PST]","[3NEG, PST, go.to]","[PROG, on.arrival]",[PST],"[PROH, ITER]","[PST, PST, PST]",[PROG],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
